In [ ]:
# | default_exp commons.loss

In [ ]:
# | export
import torch
from torchmetrics import Metric

In [ ]:
# | export
# Custom MASE Metric (since it's not built into torchmetrics)


class MASE(Metric):
    def __init__(self, input_size, horizon):
        super().__init__()
        self.input_size = input_size
        self.horizon = horizon
        self.add_state("forecast_error", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("naive_error", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("num_samples", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, y_pred, y_true, x):
        # Forecast error: mean absolute error of predictions
        forecast_error = torch.abs(y_pred - y_true).mean(dim=1)

        # Naïve seasonal error: approximate using input shift (seasonal naïve)
        naive_pred = x[:, -self.horizon :]  # Last 'horizon' points of input as naïve forecast
        naive_error = torch.abs(y_true - naive_pred).mean(dim=1)

        self.forecast_error += forecast_error.sum()
        self.naive_error += naive_error.sum()
        self.num_samples += y_true.size(0)

    def compute(self):
        if self.naive_error < 1e-8:
            return torch.tensor(float("inf"), device=self.forecast_error.device)
        return self.forecast_error / self.naive_error

In [ ]:
# | export
# Custom OWA Metric


class OWA(Metric):
    def __init__(self, smape, mase):
        super().__init__()
        self.smape = smape
        self.mase = mase
        self.add_state("smape_model", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("mase_model", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("smape_naive", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("mase_naive", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("num_samples", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, y_pred, y_true, x):
        # Model metrics
        smape_model = self.smape(y_pred, y_true)
        mase_model = self.mase(y_pred, y_true, x)

        # Naïve forecast: repeat last value of input
        naive_pred = x[:, -1:].repeat(1, y_true.size(1))
        smape_naive = self.smape(naive_pred, y_true)
        mase_naive = self.mase(naive_pred, y_true, x)

        self.smape_model += smape_model * y_true.size(0)
        self.mase_model += mase_model * y_true.size(0)
        self.smape_naive += smape_naive * y_true.size(0)
        self.mase_naive += mase_naive * y_true.size(0)
        self.num_samples += y_true.size(0)

    def compute(self):
        if self.smape_naive < 1e-8 or self.mase_naive < 1e-8:
            return torch.tensor(float("inf"), device=self.smape_model.device)
        smape_ratio = self.smape_model / self.smape_naive
        mase_ratio = self.mase_model / self.mase_naive
        return 0.5 * (smape_ratio + mase_ratio)